## Sentinel-2 cloud/shadow-free composite

This example creates, and downloads, a large cloud/shadow free composite from a year of Sentinel-2 SR imagery, over Accra, Ghana.  It demonstrates approaches for dealing with cloudy areas, and Earth Engine computational and memory limits. 

### Setup

`geemap` is required to run the notebook.  You can uncomment the cell below to install it, if it isn't installed already.

In [ ]:
# geemap should be installed if it isn't already.
# import sys
# if 'conda' in sys.prefix:
#     # install into the conda environment the notebook is being run from
#     !conda install --yes --prefix {sys.prefix} -c conda-forge geemap
# else:
#     # install into the python environment the notebook is being run from
#     !{sys.executable} -m pip install geemap

In [3]:
import logging
import ee
import geedim as gd
import geemap.foliumap as geemap
logging.basicConfig(level=logging.INFO)

# initialise earth engine with the high-volume endpoint
gd.Initialize()

INFO:root:testing


### Create and search a Sentinel-2 SR collection

This 

In [ ]:
# geojson search polygon
region = {
    'type': 'Polygon', 'coordinates': [[
    [-0.37, 5.75], [-0.37, 5.5], [0.0, 5.5], [0.0, 5.75], [-0.37, 5.75]
  ]],
}

# create and search the Sentinel-2 TOA collection
coll = gd.MaskedCollection.from_name('COPERNICUS/S2_SR')
filt_coll = coll.search(
    start_date='2019-01-01', end_date='2020-01-01', region=region, 
    cloudless_portion=80, fill_portion=30, 
    custom_filter='CLOUDY_PIXEL_PERCENTAGE<20', prob=40,
)

# print the search results
print(f'Found {len(filt_coll.properties)} images.')
print(f'Image property descriptions:\n\n{filt_coll.schema_table}\n')
print(f'Search Results:\n\n{filt_coll.properties_table}')


In [ ]:
medoid_im = filt_coll.composite('medoid', prob=40)
median_im = filt_coll.composite('medoid', prob=40)

### Visualise search result images

In [ ]:
# visualise the some search result images, and composites
s2_vis_params = dict(min=0, max=4000, bands=['B4', 'B3', 'B2'], gamma=1.5)
map = geemap.Map()
map.centerObject(ee.Geometry(region), 11)

for im_id in list(filt_coll.properties.keys())[:4]:
    im = gd.MaskedImage.from_id(im_id)
    map.addLayer(im.ee_image.clip(region), s2_vis_params, f'ID {im_id[-38:]}')

map.addLayer(medoid_im.ee_image.clip(region), s2_vis_params, 'Medoid composite')
map.addLayer(median_im.ee_image.clip(region), s2_vis_params, 'Median composite')

region_im = ee.Image().byte().paint(
    featureCollection=ee.Geometry(region), width=2, color=1
)
map.addLayer(region_im, dict(palette=['FF0000']), 'Region')

map

In [ ]:
median_im.download(
    's2_median_im.tif', region=region, crs='EPSG:32735', scale=10, 
    dtype='uint16', max_tile_size=16, overwrite=True,
)
medoid_im.download(
    's2_medoid_im.tif', region=region, crs='EPSG:32735', scale=10, 
    dtype='uint16', max_tile_size=16, overwrite=True,
)

In [ ]:
# 'geedim' = the name of a valid Earth Engine asset project
median_asset_id = f'projects/geedim/assets/s2_median_im'
medoid_asset_id = f'projects/geedim/assets/s2_medoid_im'

median_task = median_im.export(
    median_asset_id, type='asset', region=region, crs='EPSG:32735', 
    scale=10, dtype='uint16', wait=False,
)
medoid_task = medoid_im.export(
    medoid_asset_id, type='asset', region=region, crs='EPSG:32735', 
    scale=10, dtype='uint16', wait=False,
)

gd.MaskedImage.monitor_export(median_task)
gd.MaskedImage.monitor_export(medoid_task)

In [ ]:
median_asset_im = gd.MaskedImage.from_id(median_asset_id)
median_asset_im.download('s2_median_im.tif')
medoid_asset_im = gd.MaskedImage.from_id(medoid_asset_id)
medoid_asset_im.download('s2_medoid_im.tif')